# Credit Approval
Credit Approval data sourced from [here](https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/)

## Data Preprocessing
### Importing and validating dataset

In [1]:
import pandas as pd
import numpy as np
import os, ssl

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Information about attributes found at the link in crx.names file
attributeNames = ['A' + str(i+1) for i in range(16)]

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',
                   names = attributeNames,
                   header = None,
                   na_values = ['?'])

In [3]:
data.shape

(690, 16)

In [4]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [5]:
data.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [6]:
data.describe(include = np.number).round(3)

,A2,A3,A8,A11,A14,A15
count,678.000,690.000,690.000,690.000,677.000,690.000
mean,31.568,4.759,2.223,2.400,184.015,1017.386
std,11.958,4.978,3.347,4.863,173.807,5210.103
min,13.750,0.000,0.000,0.000,0.000,0.000
25%,22.602,1.000,0.165,0.000,75.000,0.000
50%,28.460,2.750,1.000,0.000,160.000,5.000
75%,38.230,7.208,2.625,3.000,276.000,395.500
max,80.250,28.000,28.500,67.000,2000.000,100000.000


In [7]:
data.describe(include = np.object)

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,678,684,684,681,681,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,468,519,519,137,399,361,395,374,625,383


### Categorical Data
Observing the categorical data types in the dataset

In [9]:
for col in data.select_dtypes(include = 'object'):
    print('Category ' + col + ' : ')
    print(data[col].value_counts(),'\n')

Category A1 : 
b    468
a    210
Name: A1, dtype: int64 

Category A4 : 
u    519
y    163
l      2
Name: A4, dtype: int64 

Category A5 : 
g     519
p     163
gg      2
Name: A5, dtype: int64 

Category A6 : 
c     137
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
r       3
Name: A6, dtype: int64 

Category A7 : 
v     399
h     138
bb     59
ff     57
z       8
j       8
dd      6
n       4
o       2
Name: A7, dtype: int64 

Category A9 : 
t    361
f    329
Name: A9, dtype: int64 

Category A10 : 
f    395
t    295
Name: A10, dtype: int64 

Category A12 : 
f    374
t    316
Name: A12, dtype: int64 

Category A13 : 
g    625
s     57
p      8
Name: A13, dtype: int64 

Category A16 : 
-    383
+    307
Name: A16, dtype: int64 



#### Redundant Features
The dataset does not contain:
- ID-like features
- Constant features 
- Date or Time features

However, columns A4 and A5 seem to have the same amount of variables, with the exact same value count distribution. This indicates there might be a repeated column.

In [11]:
A4 = data['A4'].copy()
A5 = data['A5'].copy()

A5 = A5.replace({'g':'u', 'p':'y', 'gg':'l'})

print(A4.head(10).values)
print(A5.head(10).values)

print(A4.equals(A5))

['u' 'u' 'u' 'u' 'u' 'u' 'u' 'u' 'y' 'y']
['u' 'u' 'u' 'u' 'u' 'u' 'u' 'u' 'y' 'y']
True


We can see that the columns A4 and A5 are the same, therefore, we can drop either column as to prevent redundancy.

In [12]:
data.drop(columns = 'A5', inplace=True)

### Imputing n/a Values
Let's impute numerical data with the __median__ and categorical data with their __mode__

In [13]:
data_imputed = data.copy()

for col in data_imputed.select_dtypes(include = 'object'):
    mode = data_imputed[col].mode()[0]
    print('Category ' + col + ' mode: ' + mode)
    data_imputed[col] =  data_imputed[col].fillna(mode)

Category A1 mode: b
Category A4 mode: u
Category A6 mode: c
Category A7 mode: v
Category A9 mode: t
Category A10 mode: f
Category A12 mode: f
Category A13 mode: g
Category A16 mode: -


In [14]:
for col in data_imputed.select_dtypes(include = 'number'):
    median = np.round(data_imputed[col].median(),3)
    print('Category ' + col + ' median: ' + str(median))
    data_imputed[col] =  data_imputed[col].fillna(median)

Category A2 median: 28.46
Category A3 median: 2.75
Category A8 median: 1.0
Category A11 median: 0.0
Category A14 median: 160.0
Category A15 median: 5.0


In [16]:
# Checking n/a value counts
data_imputed.isna().sum()

0

All __n/a__ values have been removed from the dataset

### Integer Encoding
Let's discretise column A2 by equal-frequency binning with "low", "medium", and "high" and then apply integer encoding.

We can also go ahead and perform a manual replace encoding for the A16 target feature, where we assume that '+' indicates a credit approval (1), and '-' indicates no approval (0).

In [17]:
data_encoded = data_imputed.copy()
data_encoded['A2'] = pd.qcut(data_encoded['A2'], q = 3, labels =['low','medium','high'])

In [18]:
data_encoded['A2'].value_counts()

medium    231
low       230
high      229
Name: A2, dtype: int64

In [19]:
level_mapping = {'low':0, 'medium':1, 'high':2 }
data_encoded['A2'] = data_encoded['A2'].replace(level_mapping)
data_encoded['A16'] = data_encoded['A16'].replace({'+':1, '-':0})

In [20]:
data_encoded.head()

,A1,A2,A3,A4,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,1,0.000,u,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,2,4.460,u,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,1,0.500,u,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,1,1.540,u,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,0,5.625,u,w,v,1.71,t,f,0,f,s,120.0,0,1


Column A2 has been equal frequency binned with 0 representing low values, 1 representing medium values and 2 representing high values.

### One Hot Encoding
Now that Category A2 has been integer encoded, we can go ahead and perform one hot encoding on the rest of the categorical features. We should separate the target feature (A16) from the rest of the dataset.

In [21]:
# Removal of the target feature, to be merged later
target = data_encoded['A16']
data_ohe = data_encoded.drop(columns = 'A16')

In [22]:
for col in data_ohe.select_dtypes(include ='object'):
    if(len(data_ohe[col].unique()) ==  2):
        data_ohe[col] = pd.get_dummies(data_ohe[col], drop_first = True)

data_ohe = pd.get_dummies(data_ohe)     

print(data_ohe.shape)

(690, 39)


In [23]:
data_ohe.head()

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
0,1,1,0.000,1.25,1,1,1,0,202.0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,2,4.460,3.04,1,1,6,0,43.0,560,...,0,1,0,0,0,0,0,1,0,0
2,0,1,0.500,1.50,1,0,0,0,280.0,824,...,0,1,0,0,0,0,0,1,0,0
3,1,1,1.540,3.75,1,1,5,1,100.0,3,...,0,0,0,0,0,1,0,1,0,0
4,1,0,5.625,1.71,1,0,0,0,120.0,0,...,0,0,0,0,0,1,0,0,0,1


### Standardised Scaling
Let's use Standard Scaling in order for all the variables to be distributed on the same scale for use in SciKitLearn models.

In [24]:
from sklearn import preprocessing

data_scaled = data_ohe.copy()

# Saving the column names prior to scaling
col_names = data_scaled.columns

# Perform standardised scaling
data_standard_scaled = preprocessing.StandardScaler().fit_transform(data_scaled)

# Convert back to a Data Frame, with correct column names
data_scaled = pd.DataFrame(data_standard_scaled,
                    columns = col_names)

print(data_scaled.shape)

data_scaled.sample(n=15, random_state = 1)

(690, 39)


,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
666,-1.511858,-1.224303,1.405407,-0.590118,-1.047504,-0.864196,-0.493887,1.087908,-0.020703,-0.195413,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
520,0.661438,-1.224303,0.551059,-0.216324,0.954650,1.157144,-0.288101,-0.919195,-0.136938,-0.150468,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
681,0.661438,0.001777,-0.755590,0.232229,-1.047504,-0.864196,-0.493887,-0.919195,-0.043950,-0.092270,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
23,-1.511858,0.001777,1.958221,0.257647,0.954650,1.157144,-0.288101,-0.919195,-0.369408,-0.193301,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
65,0.661438,1.227857,-0.647038,-0.204362,0.954650,1.157144,-0.288101,1.087908,1.955289,9.408288,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
339,0.661438,0.001777,-0.353544,-0.440600,-1.047504,-0.864196,-0.493887,1.087908,0.676706,-0.182544,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
56,0.661438,-1.224303,-0.956613,-0.639459,0.954650,-0.864196,-0.493887,-0.919195,-1.066817,-0.195413,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
394,0.661438,1.227857,-0.705334,-0.590118,-1.047504,-0.864196,-0.493887,-0.919195,-1.066817,-0.157959,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079
47,0.661438,0.001777,-0.060051,1.141296,0.954650,1.157144,0.123472,-0.919195,0.740635,-0.137791,...,-0.300079,2.0,-0.108306,-0.07636,-0.053916,-1.202834,-0.108306,0.322490,-0.108306,-0.300079
358,0.661438,0.001777,-0.353544,-0.615536,-1.047504,-0.864196,-0.493887,1.087908,-0.369408,-0.195413,...,-0.300079,-0.5,-0.108306,-0.07636,-0.053916,0.831370,-0.108306,0.322490,-0.108306,-0.300079


### Re-adding Target Feature
Let's add the target feature back to the standardised data set. The final dataset has been fully cleaned.

In [25]:
df_clean = data_scaled.assign(target = target.values)

df_clean = df_clean.round(3)

df_clean.sample(n = 15, random_state = 1)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
666,-1.512,-1.224,1.405,-0.590,-1.048,-0.864,-0.494,1.088,-0.021,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,0
520,0.661,-1.224,0.551,-0.216,0.955,1.157,-0.288,-0.919,-0.137,-0.150,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1
681,0.661,0.002,-0.756,0.232,-1.048,-0.864,-0.494,-0.919,-0.044,-0.092,...,2.0,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,0
23,-1.512,0.002,1.958,0.258,0.955,1.157,-0.288,-0.919,-0.369,-0.193,...,2.0,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,1
65,0.661,1.228,-0.647,-0.204,0.955,1.157,-0.288,1.088,1.955,9.408,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1
339,0.661,0.002,-0.354,-0.441,-1.048,-0.864,-0.494,1.088,0.677,-0.183,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,0
56,0.661,-1.224,-0.957,-0.639,0.955,-0.864,-0.494,-0.919,-1.067,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1
394,0.661,1.228,-0.705,-0.590,-1.048,-0.864,-0.494,-0.919,-1.067,-0.158,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,0
47,0.661,0.002,-0.060,1.141,0.955,1.157,0.123,-0.919,0.741,-0.138,...,2.0,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,1
358,0.661,0.002,-0.354,-0.616,-1.048,-0.864,-0.494,1.088,-0.369,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,0


__The dataset is now clean.__ It has no missing values, all columns are numerical and have been scaled, the target feature is correctly label encoded. It can now be used in a SciKitLearn model without any further data preprocessing.

### Summary and Export

In [26]:
df_clean.shape

(690, 40)

In [27]:
df_clean.describe(include='all').round(3) 

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
count,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000,...,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000,690.000
mean,-0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,...,0.000,0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.445
std,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,...,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001,0.497
min,-1.512,-1.224,-0.957,-0.665,-1.048,-0.864,-0.494,-0.919,-1.067,-0.195,...,-0.500,-0.108,-0.076,-0.054,-1.203,-0.108,-3.101,-0.108,-0.300,0.000
25%,-1.512,-1.224,-0.756,-0.616,-1.048,-0.864,-0.494,-0.919,-0.602,-0.195,...,-0.500,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,0.000
50%,0.661,0.002,-0.404,-0.366,0.955,-0.864,-0.494,-0.919,-0.137,-0.194,...,-0.500,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,0.000
75%,0.661,1.228,0.493,0.120,0.955,1.157,0.123,1.088,0.514,-0.119,...,-0.500,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1.000
max,0.661,1.228,4.672,7.858,0.955,1.157,13.294,1.088,10.557,19.012,...,2.000,9.233,13.096,18.547,0.831,9.233,0.322,9.233,3.332,1.000


In [28]:
df_clean.head(5)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,0.661,0.002,-0.957,-0.291,0.955,1.157,-0.288,-0.919,0.107,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1
1,-1.512,1.228,-0.060,0.244,0.955,1.157,0.741,-0.919,-0.817,-0.088,...,2.0,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,1
2,-1.512,0.002,-0.856,-0.216,0.955,-0.864,-0.494,-0.919,0.560,-0.037,...,2.0,-0.108,-0.076,-0.054,-1.203,-0.108,0.322,-0.108,-0.300,1
3,0.661,0.002,-0.647,0.457,0.955,1.157,0.535,1.088,-0.486,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,0.322,-0.108,-0.300,1
4,0.661,-1.224,0.174,-0.154,0.955,-0.864,-0.494,-0.919,-0.369,-0.195,...,-0.5,-0.108,-0.076,-0.054,0.831,-0.108,-3.101,-0.108,3.332,1


In [29]:
df_clean.to_csv('df_clean.csv', index = False)